# 레시피 데이터 전처리 (데이터 클렌징)

### 목적

`recipe.json` 원본 파일을 클렌징하여 `recipe_cleaned.csv` 파일을 생성합니다.

### 개발 환경 설정

1. **데이터셋 폴더 생성**

    ```bash
    cd NangPaGo/NangPaGo-data
    mkdir datasets
    ```

    `datasets` 폴더를 생성한 후, `recipe.json`과 `normalized_pattern.pkl` 파일을 해당 폴더에 넣습니다. (자료 위치: Confluence `데이터베이스/recipe 데이터셋 클렌징`)

    ```
    datasets/
    ├── normalized_pattern.pkl  # 정규화 패턴식 모음
    └── recipe.json             # 원본 데이터셋
    ```

2. **가상 환경 설정**

    ```bash
    python3.10 -m venv py310
    source py310/bin/activate
    pip install -r requirements.txt
    ```

    Python 3.10 환경에서 가상 환경을 생성하고, 필요한 라이브러리를 설치합니다.

- 필요한 파일은 `Confluence`에 `recipe.json`과 `normalized_pattern.pkl` 첨부되어 있습니다.
    ```
    datasets/
    ├── recipe_cleand.csv (저장할 데이터셋, 클렌징한 데이터)
    ├── normalized_pattern.pkl (정규화 식 모음)
    └── recipe.json (원본 데이터셋)
    ```

## 1. 데이터 전처리

### 1.1) 데이터 로드

In [49]:
import json
import pandas as pd
import numpy as np
import re
import pickle

In [50]:
# 레시피 데이터 JSON 파일 불러오기
with open("datasets/recipe.json", "r", encoding="utf-8") as fname:
    recipe_data = json.load(fname)

# 데이터 프레임 생성
df = pd.DataFrame(recipe_data)

### 1.2) 결측치 처리

- 필요없는 칼럼 제거 (Manual7 ~ Manual 20: 모든 행이 Null 값임)

In [51]:
# 빈 문자열을 None으로 대체
df.replace("", None, inplace=True)

# 결측값의 합계 확인
print(df.isnull().sum())

# 전체 행이 결측값인 컬럼 찾기
columns_to_drop = df.columns[df.isnull().sum() == len(df)]
df = df.drop(columns=columns_to_drop)

# 삭제된 컬럼 출력
print(columns_to_drop)

# 최종 DataFrame의 형태 확인
print(df.shape)

# 남은 결측값을 빈 문자열로 대체 (원상복귀)
df.fillna("", inplace=True)

RCP_PARTS_DTLS         3
RCP_WAY2               0
MANUAL_IMG20        1136
MANUAL20            1136
RCP_SEQ                0
INFO_NA                0
INFO_WGT             871
INFO_PRO               0
MANUAL_IMG13        1136
MANUAL_IMG14        1136
MANUAL_IMG15        1136
MANUAL_IMG16        1136
MANUAL_IMG10        1136
MANUAL_IMG11        1136
MANUAL_IMG12        1136
MANUAL_IMG17        1136
MANUAL_IMG18        1136
MANUAL_IMG19        1136
INFO_FAT               0
HASH_TAG             779
MANUAL_IMG02           2
MANUAL_IMG03           1
RCP_PAT2               0
MANUAL_IMG04          51
MANUAL_IMG05          54
MANUAL_IMG01           3
MANUAL01               3
ATT_FILE_NO_MK         1
MANUAL_IMG06         108
MANUAL_IMG07        1136
MANUAL_IMG08        1136
MANUAL_IMG09        1136
MANUAL08            1136
MANUAL09            1136
MANUAL06             108
MANUAL07            1136
MANUAL04              51
MANUAL05              54
MANUAL02               2
MANUAL03               1


- `RCP_PARTS_DTLS` 칼럼이 Null 인 경우 날리기

In [52]:
df[df.RCP_PARTS_DTLS == ""]

,RCP_PARTS_DTLS,RCP_WAY2,RCP_SEQ,INFO_NA,INFO_WGT,INFO_PRO,INFO_FAT,HASH_TAG,MANUAL_IMG02,MANUAL_IMG03,...,MANUAL06,MANUAL04,MANUAL05,MANUAL02,MANUAL03,ATT_FILE_NO_MAIN,INFO_CAR,RCP_NA_TIP,INFO_ENG,RCP_NM
493,,끓이기,691,577.2,,9.5,5.9,,http://www.foodsafetykorea.go.kr/uploadimg/coo...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,...,"6. 국물이 끓어 오르면 호박잎과 부추, 파, 다진 마늘을 넣고 마지막으로 달걀물을...",4. 국물낸 다슬기는 건져 달걀물과 함께 섞는다.,5. 부추는 1cm 가량의 크기로 썰고 대파는 어슷썰기 한다.,2. 호박잎은 줄기를 꺾으며 잡아당겨 실 같은 껍질을 벗기고 씻는다.,3. 멸치다시마국물 1컵에 된장을 풀고 씻은 다슬기살을 함께 넣어 중불에서 5분정도...,http://www.foodsafetykorea.go.kr/uploadimg/201...,6.2,호박잎은 비벼서 연하게 만든 후 사용한다. - 다슬기를 달걀물에 묻혀 넣으면 달걀이...,86.7,호박잎다슬기된장국
494,,찌기,692,478.8,,10.7,7.2,,http://www.foodsafetykorea.go.kr/uploadimg/coo...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,...,"6. 소스는 다진 양파와 마늘을 먼저 볶아 향을 낸 후 토마토케첩, 간장, 매실액을...","4. 다진 돼지고기에 파, 다진 마늘, 참기름, 설탕, 소금, 후춧가루를 넣고 양념...",5. 찐 양배추에 밀가루를 뿌리고 3의 재료를 놓고 돌돌 감아 찜통에 찐다.,2. 자른 양배추는 김이 오른 찜통에 5분 정도 찐다.,"3. 당근, 양파, 표고버섯은 곱게 다지고 두부는 으깨 면보에 짠다",http://www.foodsafetykorea.go.kr/uploadimg/201...,14.1,양배추의 단단한 잎줄기는 부드러운 잎과 열을 받는 시차가 다르고 음식을 할 때도 모...,165,양배추두부찜과 양파케첩소스
1029,,끓이기,831,210,,7.5,3.6,,http://www.foodsafetykorea.go.kr/uploadimg/coo...,http://www.foodsafetykorea.go.kr/uploadimg/coo...,...,6. 나머지 재료를 넣어 연하게 졸여낸다,"4. 마늘쫑은 3cm 길이로 자르고, 파프리카는 1.5 x 4cm로 썬다.",5. 냄비에 저나트륨 조림소스 재료를 넣고 곤약을 넣어 약한 불에서 졸인다.,2. 곤약은 데쳐서 준비한다.,3. 콩은 불리고 마늘은 편으로 썬다.,http://www.foodsafetykorea.go.kr/uploadimg/201...,10.7,저나트륨 조림소스는 보통 조림소스보다 간장 양을 1/3정도 줄여서 사용하므로 색을 ...,96.4,곤약 콩조림


In [53]:
df = df[df['RCP_PARTS_DTLS'] != ""]
df.shape

(1133, 27)

### 1.3) 데이터 타입 변환

In [54]:
numeric_columns = ["RCP_SEQ", "INFO_ENG", "INFO_CAR", "INFO_PRO", "INFO_FAT", "INFO_NA"]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")
df.dtypes

RCP_PARTS_DTLS       object
RCP_WAY2             object
RCP_SEQ               int64
INFO_NA             float64
INFO_WGT             object
INFO_PRO            float64
INFO_FAT            float64
HASH_TAG             object
MANUAL_IMG02         object
MANUAL_IMG03         object
RCP_PAT2             object
MANUAL_IMG04         object
MANUAL_IMG05         object
MANUAL_IMG01         object
MANUAL01             object
ATT_FILE_NO_MK       object
MANUAL_IMG06         object
MANUAL06             object
MANUAL04             object
MANUAL05             object
MANUAL02             object
MANUAL03             object
ATT_FILE_NO_MAIN     object
INFO_CAR            float64
RCP_NA_TIP           object
INFO_ENG            float64
RCP_NM               object
dtype: object

### 1.4) 행 순서 정렬 및 컬럼명 변경

#### RCP_SEQ 기준으로 정렬

In [55]:
df = df.sort_values(by="RCP_SEQ", ascending=True).reset_index(drop=True)

#### 컬럼명 변경

In [56]:
column_mapping = {
    "RCP_SEQ": "ID",
    'RCP_NM': "NAME",
    'RCP_WAY2': "COOKING_METHOD",
    'RCP_PAT2': "CATEGORY",
    'INFO_WGT': "WEIGHT",
    'INFO_ENG': "CALORIE",
    'INFO_CAR': "CARBOHYDRATE",
    'INFO_PRO': "PROTEIN",
    'INFO_FAT': "FAT",
    'INFO_NA': "NATRIUM",
    'HASH_TAG': "MAIN_INGREDIENT",
    'ATT_FILE_NO_MAIN': "MAIN_IMAGE_LOW",
    'ATT_FILE_NO_MK': "MAIN_IMAGE",
    'RCP_PARTS_DTLS': "INGREDIENT_DETAIL",
    'MANUAL01': "MANUAL_01",
    'MANUAL_IMG01': "MANUAL_IMAGE_01",
    'MANUAL02': "MANUAL_02",
    'MANUAL_IMG02': "MANUAL_IMAGE_02",
    'MANUAL03': "MANUAL_03",
    'MANUAL_IMG03': "MANUAL_IMAGE_03",
    'MANUAL04': "MANUAL_04",
    'MANUAL_IMG04': "MANUAL_IMAGE_04",
    'MANUAL05': "MANUAL_05",
    'MANUAL_IMG05': "MANUAL_IMAGE_05",
    'MANUAL06': "MANUAL_06",
    'MANUAL_IMG06': "MANUAL_IMAGE_06",
    'RCP_NA_TIP': "RECIPE_DESCRIPTION"
}

# 열 이름 변경
df = df.rename(columns=column_mapping).reset_index(drop=True).drop('ID', axis=1)

## 2. `INGREDIENT_DETAIL` 칼럼 데이터 클렌징 (`recipe_cleaned` 생성)

### 2.1) `INGREDIENT_DETAIL` 칼럼을 대상으로 불용어 제거
- `[1인분]`, `1인분 기준<br>\n[주재료]` `•필수 재료 :` 등 불필요한 용어 제거 및 순수 재료 데이터로 정제

In [ ]:
# df['INGREDIENT_ORIGINAL'] = df['INGREDIENT_DETAIL']

In [58]:
df['INGREDIENT_DETAIL'] = df['INGREDIENT_DETAIL'].apply(
    lambda x: x.replace('청·홍', '청홍') \
                .replace('L.A', 'LA') \
                .replace('올ㄹ브오일', '올리브오일') \
                .replace('머스터드조스', '머스터드소스') \
                .replace('후추적당량[양송이 속양파다진것', '후추적당량, 양송이, 양파다진것') \
                .replace('m 적양배추', ', 적양배추') \
                .replace('l고추장', ' 고추장') \
    if isinstance(x, str) else x
)

In [59]:
with open("datasets/normalized_pattern.pkl", "rb") as f:
    patterns = pickle.load(f)

In [60]:
def split_with_protection(text):
    # 괄호 안의 쉼표 보호
    text = re.sub(r'\([^)]*\)', lambda m: m.group(0).replace(',', ' '), text)
    # 쉼표로 분리
    items = text.split(',')
    return items

def extract_food_names_from_list(text_list, pattern):
    food_names_list = []
    for origin in text_list:
        text = origin
        if bool(re.search(pattern[0], text)):
            text = re.sub(pattern[0], pattern[1], text)
        
        items = split_with_protection(text)
        items = [item for item in items if item.strip()]
        
        food_names = []
        for item in items:
            item = item.strip()
            if item:
                food_names.append(item)
        
        food_str = ", ".join(food_names)
        food_names_list.append(food_str)
    
    return food_names_list

def contains_food_info(text, pattern):
    return bool(re.search(pattern, text))

In [61]:
ingredient_detail_series = df['INGREDIENT_DETAIL']

for i, pattern in enumerate(patterns):
    food_names = extract_food_names_from_list(ingredient_detail_series, pattern)
    ingredient_detail_series = pd.Series(food_names) # Series로 저장

df['INGREDIENT_DETAIL'] = ingredient_detail_series # 정규화된 재료 정보를 다시 DataFrame에 저장

### 2.2) `INGREDIENT_DETAIL`에서 `NAME`과 `AMOUNT`로 분리

In [62]:
def split_ingredients(ingredients):
    items = ingredients.split(",")
    ingredient_names = []
    ingredient_amounts = []
    
    for item in items:
        item = item.strip()
        # 숫자, '약간', '(약간)', '(숫자)' 패턴을 고려한 정규식
        match = re.match(r"(.+?)(\s*[\d⅓½¼¾⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞]+.*|\s*약간.*|\s*\(약간\).*|\s*\([\d⅓½¼¾⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞]+.*?\).*)$", item)
        if match:
            ingredient, amount = match.group(1), match.group(2)
            ingredient_names.append(ingredient.strip())
            ingredient_amounts.append(amount.strip())
        else:
            ingredient_names.append(item.strip())
            ingredient_amounts.append("")
    
    return pd.Series([ingredient_names, ingredient_amounts])

# 재료 이름과 양 분리
df[['INGREDIENT_NAME', 'INGREDIENT_AMOUNT']] = ingredient_detail_series.apply(split_ingredients)

# 결과 출력 (확인용)
print(df[['INGREDIENT_DETAIL', 'INGREDIENT_NAME', 'INGREDIENT_AMOUNT']])

                                      INGREDIENT_DETAIL  \
0     새송이버섯 100g(3개), 올리브유 10g(2작은술), 두유 20g(1⅓큰술), ...   
1     백일송이버섯 65g(1컵), 노랑 파프리카 5g(5×1cm), 빨강 파프리카 5g(...   
2     가지(15g), 새송이버섯(15g), 표고버섯(10g), 땅콩가루(5g), 다진마늘...   
3     연두부 75g(3/4모), 칵테일새우 20g(5마리), 달걀 30g(1/2개), 생...   
4     조선부추 50g, 날콩가루 7g(1⅓작은술), 저염간장 3g(2/3작은술), 다진 ...   
...                                                 ...   
1128  소고기(치맛살)(100g), 김(6g(3장)), 튀김가루(20g), 식용유(45g)...   
1129  감태 (2g(2장)), 라이스페이퍼(60g), 양상추(80g), 적채(20g), 건...   
1130  부추(30g), 양파(10g), 배(15g), 다진 마늘(8g), 고춧가루(8g),...   
1131  열무(30g), 톳(10g), 마늘(3g), 생강(1g), 어간장(5g), 고춧가루...   
1132  알배추(23g), 쪽파(5g), 통깨(2g), 겉절이 양념(15g), 찐 감자(20...   

                                        INGREDIENT_NAME  \
0                [새송이버섯, 올리브유, 두유, 생크림, 청양고추, 후추, 새싹채소]   
1                      [백일송이버섯, 노랑 파프리카, 빨강 파프리카, 카놀라유]   
2              [가지, 새송이버섯, 표고버섯, 땅콩가루, 다진마늘, 참기름, 저염간장]   
3                  [연두부, 칵테일새우, 달걀, 생크림, 설탕, 무염버터, 시금치]

### 2.3) `recipe_cleaned.csv` 파일 저장

In [63]:
df.index = range(1, len(df) + 1)
df.index.name = 'ID'

In [64]:
df.to_csv("datasets/recipe_cleand.csv", index=True, encoding="utf-8")